# Support Vector Machine Learning Model (95% variance, with scaling - F1 Macro)

Import required packages

In [ ]:
import numpy as np # for multi-dimensional array operations
import pandas as pd # for reading data from .csv files
from sklearn.svm import SVC # for support vector machine model
from sklearn.decomposition import PCA # for principle component analysis (dimensionality reduction)
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, StratifiedKFold # for getting the best hyper parameters
from sklearn.preprocessing import MinMaxScaler # for scaling of data before PCA

Assign the training set and testing set to variables for easy reference

In [ ]:
train_set = pd.read_csv('../train_tfidf_features.csv') # import the training set
test_set = pd.read_csv('../test_tfidf_features.csv') # import the testing set

Principal Component Analysis for train_set (95% variance, with scaling)

In [ ]:
train_set_label = train_set.loc[:, ["label"]]
features_names = [str(i) for i in range(0, 5000)]
train_set_features = train_set.loc[:, features_names] # train_set_features will not contain the label and id columns

# scale the dataset before PCA
scaler = MinMaxScaler()
train_set_rescaled = scaler.fit_transform(train_set_features)

# perform PCA
pca = PCA(n_components = 0.95)
train_set_reduced = pca.fit_transform(train_set_rescaled)
train_set_reduced = pd.DataFrame(data = train_set_reduced)
train_set_reduced

In [ ]:
X = train_set_reduced
y = train_set_label

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) # Train a SVC model using different kernel
X_train = X
y_train = y

In [ ]:
X_train

In [ ]:
y_train

Principal Component Analysis for test_set (95% variance)

In [ ]:
features_names = [str(i) for i in range(0, 5000)]
test_set_features = test_set.loc[:, features_names] # test_set_features will not contain the label and id columns

# scale the dataset before PCA
test_set_rescaled = scaler.transform(test_set_features)

# perform PCA
test_set_reduced = pca.transform(test_set_rescaled) # use the pca from the train_set?
test_set_features = pd.DataFrame(data = test_set_reduced)
test_set_features

Tuning the hyper-parameters and training the model based on the best hyper-parameters

In [ ]:
# hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear', 'poly', 'sigmoid', 'rbf']} # initialise the hyper-parameters
hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']} # initialise the hyper-parameters
kfold = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0) # for 3-fold cross validation
# grid = GridSearchCV(SVC(), hyper_parameters, refit = True, verbose = 2) # cretae a GridSearchCV object to git to the taining data
grid = GridSearchCV(SVC(), param_grid = hyper_parameters, scoring = 'f1_macro', refit = 'f1_macro', n_jobs = 1 , cv = kfold, verbose = 2)
grid.fit(X_train, np.ravel(y_train)) # training the model using the best hyper-parameters
print(grid.best_params_) # gets the best hyper-parameters for SVM

Predicting the labels for the test dataset based on the model with the best hyper-parameters

In [ ]:
y_predicted = grid.predict(test_set_features)

In [ ]:
# y_predicted = svc_model.predict(test_set_features)
y_predicted = pd.DataFrame(y_predicted, columns = ['label']) # convert y_predicted from nparray to pandas dataframe
y_predicted.insert(loc = 0, column = 'id', value = [i for i in range(17185, 17185 + 4296)]) # insert a column of the ids, starting from 17185
y_predicted.to_csv('skynet_submission_95_with_scaling_f1macro.csv', index = False) # output the predicted labels to ./skynet_submission_95_with_scaling_f1macro.csv